In [16]:
!pip install fuzzywuzzy

In [8]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Load OCR-extracted patents
ocr_patents_file = "ocr_extracted_patents.xlsx"  
ocr_df = pd.read_excel(ocr_patents_file)

# Load the official patent database
patent_db_file = "patent_database.xlsx"
db_df = pd.read_excel(patent_db_file)

# Step 1: Group patentees by Patent ID (combine multiple patentees into one string)
db_grouped = db_df.groupby("Patent ID").agg({
    "Patentee name": lambda x: ", ".join(x.unique()),  # Merge multiple patentees into one row
    "Patent title": "first"  # Patent title is the same for all duplicates
}).reset_index()

# Standardize text: Lowercase & remove extra spaces
def clean_text(text):
    if pd.isna(text):
        return ""
    return " ".join(str(text).lower().strip().split())

ocr_df["Cleaned Patentee"] = ocr_df["Patentee"].apply(clean_text)
ocr_df["Cleaned Title"] = ocr_df["Patent Title"].apply(clean_text)
db_grouped["Cleaned Patentee"] = db_grouped["Patentee name"].apply(clean_text)
db_grouped["Cleaned Title"] = db_grouped["Patent title"].apply(clean_text)

# Function to get best match from database
def best_fuzzy_match(ocr_patentee, ocr_title, db_df, threshold=60):
    best_match = None
    best_score = 0
    best_patent_id = None

    # Iterate through all patents in the database
    for _, row in db_df.iterrows():
        db_patentee = row["Cleaned Patentee"]
        db_title = row["Cleaned Title"]
        patent_id = row["Patent ID"]

        # Compute similarity scores
        patentee_score = fuzz.token_sort_ratio(ocr_patentee, db_patentee)
        title_score = fuzz.token_sort_ratio(ocr_title, db_title)

        # Weighted score (more weight to patentee match)
        combined_score = (patentee_score * 0.6) + (title_score * 0.4)

        # Keep the best match
        if combined_score > best_score and combined_score >= threshold:
            best_score = combined_score
            best_match = (db_patentee, db_title)
            best_patent_id = patent_id

    return best_patent_id, best_match, best_score

# Apply best fuzzy matching for each OCR patent
ocr_df[["Matched Patent ID", "Best Match", "Match Score"]] = ocr_df.apply(
    lambda row: best_fuzzy_match(row["Cleaned Patentee"], row["Cleaned Title"], db_grouped), axis=1, result_type="expand"
)

# Merge matched results back with the grouped database
merged_df = ocr_df.merge(db_grouped, left_on="Matched Patent ID", right_on="Patent ID", how="left", suffixes=("_OCR", "_DB"))



### V2

In [36]:
import pandas as pd
import re
from fuzzywuzzy import fuzz

# Load OCR-extracted patents
ocr_patents_file = "ocr_extracted_patents.xlsx"  
ocr_df = pd.read_excel(ocr_patents_file)

# Load the official patent database
patent_db_file = "patent_database.xlsx"
db_df = pd.read_excel(patent_db_file)

# Step 1: Group patentees by Patent ID (combine multiple patentees into one string)
db_grouped = db_df.groupby("Patent ID").agg({
    "Patentee name": lambda x: ", ".join(x.unique()),  # Merge multiple patentees into one row
    "Patent title": "first",  # Patent title is the same for all duplicates
    "Issue date": "first"  # Keep the first issue date for each Patent ID
}).reset_index()

# French month mapping
french_months = {
    "janvier": "01", "février": "02", "mars": "03", "avril": "04", "mai": "05", "juin": "06",
    "juillet": "07", "août": "08", "septembre": "09", "octobre": "10", "novembre": "11", "décembre": "12"
}

def convert_ocr_date(ocr_date):
    """
    Converts OCR-extracted date from '18 février 1796' to 'YYYY-MM-DD'.
    Returns an empty string ("") if the date is not recognized.
    """
    match = re.search(r"(\d{1,2})\s+([a-zéû]+)\s+(\d{4})", str(ocr_date), re.IGNORECASE)
    if match:
        day, month_name, year = match.groups()
        month = french_months.get(month_name.lower())  # Convert month name to number
        if month:
            return f"{year}-{month}-{int(day):02d}"  # Convert to YYYY-MM-DD
    return ""  # If the date is not recognizable, return an empty string

def convert_db_date(db_date):
    """
    Converts a database Issue Date from 'DD/MM/YYYY' to 'YYYY-MM-DD'.
    Returns an empty string ("") if conversion fails.
    """
    try:
        day, month, year = db_date.split("/")
        return f"{year}-{month}-{int(day):02d}"
    except Exception:
        return ""  # Return empty string for invalid dates

# Standardize text: Lowercase & remove extra spaces
def clean_text(text):
    if pd.isna(text):
        return ""
    return " ".join(str(text).lower().strip().split())

ocr_df["Cleaned Patentee"] = ocr_df["Patentee"].apply(clean_text)
ocr_df["Cleaned Title"] = ocr_df["Patent Title"].apply(clean_text)
db_grouped["Cleaned Patentee"] = db_grouped["Patentee name"].apply(clean_text)
db_grouped["Cleaned Title"] = db_grouped["Patent title"].apply(clean_text)

# Apply date conversion functions
ocr_df["Cleaned Date"] = ocr_df["Extracted Date"].apply(lambda x: convert_ocr_date(str(x)) if pd.notna(x) else "")
db_grouped["Cleaned Date"] = db_grouped["Issue date"].apply(lambda x: convert_db_date(str(x)) if pd.notna(x) else "")

# Step 3: Define fuzzy matching function
def best_fuzzy_match(ocr_patentee, ocr_title, ocr_date, db_df, threshold=60):
    best_match = None
    best_score = 0
    best_patent_id = None

    for _, row in db_df.iterrows():
        db_patentee = row["Cleaned Patentee"]
        db_title = row["Cleaned Title"]
        db_date = row["Cleaned Date"]
        patent_id = row["Patent ID"]

        # Compute similarity scores
        patentee_score = fuzz.token_sort_ratio(ocr_patentee, db_patentee)
        title_score = fuzz.token_sort_ratio(ocr_title, db_title)

        # Handle missing/unrecognized dates
        if ocr_date and db_date:  # Compute date score if both are available
            date_score = fuzz.ratio(ocr_date, db_date)
            date_weight = 0.2
        else:
            date_score = 0  # No contribution if one or both dates are missing
            date_weight = 0  # Ignore date weight

        # Weighted score calculation
        combined_score = (patentee_score * 0.5) + (title_score * 0.3) + (date_score * date_weight)

        # Keep the best match
        if combined_score > best_score and combined_score >= threshold:
            best_score = combined_score
            best_match = (db_patentee, db_title, db_date)
            best_patent_id = patent_id

    return best_patent_id, best_match, best_score

# Step 4: Apply best fuzzy matching for each OCR patent
ocr_df[["Matched Patent ID", "Best Match", "Match Score"]] = ocr_df.apply(
    lambda row: best_fuzzy_match(row["Cleaned Patentee"], row["Cleaned Title"], row["Cleaned Date"], db_grouped),
    axis=1,
    result_type="expand"
)

# Step 5: Merge matched results back with the grouped database
merged_df = ocr_df.merge(db_grouped, left_on="Matched Patent ID", right_on="Patent ID", how="left", suffixes=("_OCR", "_DB"))

# Save final output
#merged_df.to_excel("matched_patents.xlsx", index=False)

#print("Fuzzy matching completed. Results saved to 'matched_patents.xlsx'.")


In [37]:
merged_df

,Patent Type,Patent Category,Duration (Years),Patent Title,Patentee,Date Line,Extracted Date,Word Count,Character Count,Page Number,...,Matched Patent ID,Best Match,Match Score,Patent ID,Patentee name,Patent title,Issue date,Cleaned Patentee_DB,Cleaned Title_DB,Cleaned Date_DB
0,BREVET,INVENTION,CINQ,des procédés relatifs au laminage du fer et de,Jamar père et fils,4 septembre 1791.,4 septembre 1791,2521,14005,12,...,b1169,"(jamain père et fils, charles-robert et ponce-...",65.1,b1169,"Jamain père et fils, Charles-Robert et Ponce-N...",Procédés relatifs au laminage du fer et de l'a...,4/09/1791,"jamain père et fils, charles-robert et ponce-n...",procédés relatifs au laminage du fer et de l'a...,1791-09-04
1,BREVET,INVENTION,CINQ,un mécanisme propre à rendre régulière la fabri-,FureT - LapouLaYE,18 février 1796.,18 février 1796,231,1328,20,...,b1943,"(furet-laboulaye martin, machine à adapter à t...",64.7,b1943,Furet-Laboulaye Martin,Machine à adapter à toute sorte de métiers à t...,18/02/1794,furet-laboulaye martin,machine à adapter à toute sorte de métiers à t...,1794-02-18
2,BREVET,IMPORTATION,DIX,des bretelles et ceintures élastiques,John WaAuxer fils,17 GAOÎÛL 1791.,Unrecognized Date,507,2724,21,...,b236,"(walker john, bretelles et ceintures élastique...",60.2,b236,Walker John,Bretelles et ceintures élastiques,17/08/1791,walker john,bretelles et ceintures élastiques,1791-08-17
3,BREVET,INVENTION,DIX,la construction de nouvelles grues,Baunrvrry-LiavA,12 décembre 1791.,12 décembre 1791,24,156,27,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BREVET,INVENTION,DIX,un peson ou romaine à cadran,HAmAzN père et fils,9 avril 1792.,9 avril 1792,38,214,27,...,b1508,"(hanin père et fils, françois et françois, pes...",61.5,b1508,"Hanin père et fils, François et François","Peson ou romaine à cadran, propre à peser, san...",9/04/1792,"hanin père et fils, françois et françois","peson ou romaine à cadran, propre à peser, san...",1792-04-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,BREVET,PERFECTIONNEMENT,CINQ,une machine à tondre les draps,LesLanc ParoissieN,"25 octobre 1803, et 27 juin 1806.",25 octobre 1803,699,3852,269,...,b2004,"(leblanc-paroissien françois, machine à tondre...",86.1,b2004,Leblanc-Paroissien François,Machine à tondre les draps,25/10/1803,leblanc-paroissien françois,machine à tondre les draps,1803-10-25
75,BREVET,INVENTION,DIX,des procédés relatifs à la teinture et à l’imp...,Dorrus,17 novembre 1798.,17 novembre 1798,1647,9341,272,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,BREVET,INVENTION,CINQ,une cuisine économique,CoursAuT,ao décembre 1803.,Unrecognized Date,1036,5761,285,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,BREVET,INVENTION,QUINZE,un instrument nommé /le prompi Calculateur,John Firca,21 décembre 1793.,21 décembre 1793,584,3190,289,...,b291,"(fitch john, instrument nommé le prompt calcul...",86.1,b291,Fitch John,Instrument nommé le prompt calculateur marin,21/12/1793,fitch john,instrument nommé le prompt calculateur marin,1793-12-21


In [38]:
# Print only rows where "Matched Patent ID" is not null (successfully matched rows)
matched_rows = merged_df[merged_df["Matched Patent ID"].notna()]


In [39]:
matched_rows

,Patent Type,Patent Category,Duration (Years),Patent Title,Patentee,Date Line,Extracted Date,Word Count,Character Count,Page Number,...,Matched Patent ID,Best Match,Match Score,Patent ID,Patentee name,Patent title,Issue date,Cleaned Patentee_DB,Cleaned Title_DB,Cleaned Date_DB
0,BREVET,INVENTION,CINQ,des procédés relatifs au laminage du fer et de,Jamar père et fils,4 septembre 1791.,4 septembre 1791,2521,14005,12,...,b1169,"(jamain père et fils, charles-robert et ponce-...",65.1,b1169,"Jamain père et fils, Charles-Robert et Ponce-N...",Procédés relatifs au laminage du fer et de l'a...,4/09/1791,"jamain père et fils, charles-robert et ponce-n...",procédés relatifs au laminage du fer et de l'a...,1791-09-04
1,BREVET,INVENTION,CINQ,un mécanisme propre à rendre régulière la fabri-,FureT - LapouLaYE,18 février 1796.,18 février 1796,231,1328,20,...,b1943,"(furet-laboulaye martin, machine à adapter à t...",64.7,b1943,Furet-Laboulaye Martin,Machine à adapter à toute sorte de métiers à t...,18/02/1794,furet-laboulaye martin,machine à adapter à toute sorte de métiers à t...,1794-02-18
2,BREVET,IMPORTATION,DIX,des bretelles et ceintures élastiques,John WaAuxer fils,17 GAOÎÛL 1791.,Unrecognized Date,507,2724,21,...,b236,"(walker john, bretelles et ceintures élastique...",60.2,b236,Walker John,Bretelles et ceintures élastiques,17/08/1791,walker john,bretelles et ceintures élastiques,1791-08-17
4,BREVET,INVENTION,DIX,un peson ou romaine à cadran,HAmAzN père et fils,9 avril 1792.,9 avril 1792,38,214,27,...,b1508,"(hanin père et fils, françois et françois, pes...",61.5,b1508,"Hanin père et fils, François et François","Peson ou romaine à cadran, propre à peser, san...",9/04/1792,"hanin père et fils, françois et françois","peson ou romaine à cadran, propre à peser, san...",1792-04-09
5,BREVET,PERFECTIONNEMENT,DIX,la construction d’une machine hydraulique,DesmAREST,13 septembre 1792.,13 septembre 1792,20,141,27,...,b1066,"(desmarest jean-baptiste, machine hydraulique ...",64.9,b1066,Desmarest Jean-Baptiste,Machine hydraulique propre à élever l'eau,13/09/1793,desmarest jean-baptiste,machine hydraulique propre à élever l'eau,1793-09-13
7,BREVET,INVENTION,CINQ,la manière de convertir des planches formées,GATTEAUX,17 février 1798.,17 février 1798,623,3623,29,...,b1942,"(gatteaux, presses mécaniques propres à appose...",71.3,b1942,Gatteaux,Presses mécaniques propres à apposer d'un seul...,11/05/1813,gatteaux,presses mécaniques propres à apposer d'un seul...,1813-05-11
9,BREVET,INVENTION,CINQ,la construction d’un gril aérien,ScHmIDT,28 janvier 1799.,28 janvier 1799,1046,5803,32,...,b1006,"(schmidt nicolas, nouvelle machine désignée so...",61.5,b1006,Schmidt Nicolas,Nouvelle machine désignée sous le nom de gril ...,17/01/1799,schmidt nicolas,nouvelle machine désignée sous le nom de gril ...,1799-01-17
13,BREVET,INVENTION,CINQ,des procédés relatifs à la préparation des cuirs,BAUMANN,18 mai 1799.,18 mai 1799,76,445,58,...,b585,"(hullot, baumann, procédés relatifs à la prépa...",70.0,b585,"Hullot, Baumann",Procédés relatifs à la préparation des cuirs e...,18/05/1799,"hullot, baumann",procédés relatifs à la préparation des cuirs e...,1799-05-18
15,BREVET,INVENTION,CINQ,un bâtiment propre à naviguer sans voiles,DecAcROIX,"27 juillet 1799,",27 juillet 1799,32,187,59,...,b1371,"(delacroix etienne-laurent, bâtiment propre à ...",63.4,b1371,Delacroix Etienne-Laurent,"Bâtiment propre à naviguer sans voiles, sans c...",12/07/1799,delacroix etienne-laurent,"bâtiment propre à naviguer sans voiles, sans c...",1799-07-12
16,BREVET,INVENTION,CINQ,un manége de campagﬂe ou pôrtatif,Focarn-CHATEAU,20 juillet 1799.,20 juillet 1799,362,1961,60,...,b1254,"(focard-château louis-jean, manège de campagne...",74.1,b1254,Focard-Château Louis-Jean,Manège de campagne,20/07/1799,focard-château louis-jean,manège de campagne,1799-07-20


In [40]:
len(matched_rows)

44

In [42]:
df_filtered = matched_rows[["Patent title", "Patentee", "Extracted Date", "Patentee name", "Patent Title", "Issue date"]]


In [43]:
df_filtered

,Patent title,Patentee,Extracted Date,Patentee name,Patent Title,Issue date
0,Procédés relatifs au laminage du fer et de l'a...,Jamar père et fils,4 septembre 1791,"Jamain père et fils, Charles-Robert et Ponce-N...",des procédés relatifs au laminage du fer et de,4/09/1791
1,Machine à adapter à toute sorte de métiers à t...,FureT - LapouLaYE,18 février 1796,Furet-Laboulaye Martin,un mécanisme propre à rendre régulière la fabri-,18/02/1794
2,Bretelles et ceintures élastiques,John WaAuxer fils,Unrecognized Date,Walker John,des bretelles et ceintures élastiques,17/08/1791
4,"Peson ou romaine à cadran, propre à peser, san...",HAmAzN père et fils,9 avril 1792,"Hanin père et fils, François et François",un peson ou romaine à cadran,9/04/1792
5,Machine hydraulique propre à élever l'eau,DesmAREST,13 septembre 1792,Desmarest Jean-Baptiste,la construction d’une machine hydraulique,13/09/1793
7,Presses mécaniques propres à apposer d'un seul...,GATTEAUX,17 février 1798,Gatteaux,la manière de convertir des planches formées,11/05/1813
9,Nouvelle machine désignée sous le nom de gril ...,ScHmIDT,28 janvier 1799,Schmidt Nicolas,la construction d’un gril aérien,17/01/1799
13,Procédés relatifs à la préparation des cuirs e...,BAUMANN,18 mai 1799,"Hullot, Baumann",des procédés relatifs à la préparation des cuirs,18/05/1799
15,"Bâtiment propre à naviguer sans voiles, sans c...",DecAcROIX,27 juillet 1799,Delacroix Etienne-Laurent,un bâtiment propre à naviguer sans voiles,12/07/1799
16,Manège de campagne,Focarn-CHATEAU,20 juillet 1799,Focard-Château Louis-Jean,un manége de campagﬂe ou pôrtatif,20/07/1799


In [41]:
# Save results to Excel for review
output_file = "matched_patents_with_ID.xlsx"
merged_df.to_excel(output_file, index=False)


### Vol 12

In [1]:
import pandas as pd
import re
from fuzzywuzzy import fuzz

# Load OCR-extracted patents
ocr_patents_file = "ocr_extracted_patents_12_BW.xlsx"  
ocr_df = pd.read_excel(ocr_patents_file)

# Load the official patent database
patent_db_file = "patent_database.xlsx"
db_df = pd.read_excel(patent_db_file)

# Step 1: Group patentees by Patent ID (combine multiple patentees into one string)
db_grouped = db_df.groupby("Patent ID").agg({
    "Patentee name": lambda x: ", ".join(x.unique()),  # Merge multiple patentees into one row
    "Patent title": "first",  # Patent title is the same for all duplicates
    "Issue date": "first"  # Keep the first issue date for each Patent ID
}).reset_index()

# French month mapping
french_months = {
    "janvier": "01", "février": "02", "mars": "03", "avril": "04", "mai": "05", "juin": "06",
    "juillet": "07", "août": "08", "septembre": "09", "octobre": "10", "novembre": "11", "décembre": "12"
}

def convert_ocr_date(ocr_date):
    """
    Converts OCR-extracted date from '18 février 1796' to 'YYYY-MM-DD'.
    Returns an empty string ("") if the date is not recognized.
    """
    match = re.search(r"(\d{1,2})\s+([a-zéû]+)\s+(\d{4})", str(ocr_date), re.IGNORECASE)
    if match:
        day, month_name, year = match.groups()
        month = french_months.get(month_name.lower())  # Convert month name to number
        if month:
            return f"{year}-{month}-{int(day):02d}"  # Convert to YYYY-MM-DD
    return ""  # If the date is not recognizable, return an empty string

def convert_db_date(db_date):
    """
    Converts a database Issue Date from 'DD/MM/YYYY' to 'YYYY-MM-DD'.
    Returns an empty string ("") if conversion fails.
    """
    try:
        day, month, year = db_date.split("/")
        return f"{year}-{month}-{int(day):02d}"
    except Exception:
        return ""  # Return empty string for invalid dates

# Standardize text: Lowercase & remove extra spaces
def clean_text(text):
    if pd.isna(text):
        return ""
    return " ".join(str(text).lower().strip().split())

ocr_df["Cleaned Patentee"] = ocr_df["Patentee"].apply(clean_text)
ocr_df["Cleaned Title"] = ocr_df["Patent Title"].apply(clean_text)
db_grouped["Cleaned Patentee"] = db_grouped["Patentee name"].apply(clean_text)
db_grouped["Cleaned Title"] = db_grouped["Patent title"].apply(clean_text)

# Apply date conversion functions
ocr_df["Cleaned Date"] = ocr_df["Extracted Date"].apply(lambda x: convert_ocr_date(str(x)) if pd.notna(x) else "")
db_grouped["Cleaned Date"] = db_grouped["Issue date"].apply(lambda x: convert_db_date(str(x)) if pd.notna(x) else "")

# Step 3: Define fuzzy matching function
def best_fuzzy_match(ocr_patentee, ocr_title, ocr_date, db_df, threshold=60):
    best_match = None
    best_score = 0
    best_patent_id = None

    for _, row in db_df.iterrows():
        db_patentee = row["Cleaned Patentee"]
        db_title = row["Cleaned Title"]
        db_date = row["Cleaned Date"]
        patent_id = row["Patent ID"]

        # Compute similarity scores
        patentee_score = fuzz.token_sort_ratio(ocr_patentee, db_patentee)
        title_score = fuzz.token_sort_ratio(ocr_title, db_title)

        # Handle missing/unrecognized dates
        if ocr_date and db_date:  # Compute date score if both are available
            date_score = fuzz.ratio(ocr_date, db_date)
            date_weight = 0.2
        else:
            date_score = 0  # No contribution if one or both dates are missing
            date_weight = 0  # Ignore date weight

        # Weighted score calculation
        combined_score = (patentee_score * 0.5) + (title_score * 0.3) + (date_score * date_weight)

        # Keep the best match
        if combined_score > best_score and combined_score >= threshold:
            best_score = combined_score
            best_match = (db_patentee, db_title, db_date)
            best_patent_id = patent_id

    return best_patent_id, best_match, best_score

# Step 4: Apply best fuzzy matching for each OCR patent
ocr_df[["Matched Patent ID", "Best Match", "Match Score"]] = ocr_df.apply(
    lambda row: best_fuzzy_match(row["Cleaned Patentee"], row["Cleaned Title"], row["Cleaned Date"], db_grouped),
    axis=1,
    result_type="expand"
)

# Step 5: Merge matched results back with the grouped database
merged_df = ocr_df.merge(db_grouped, left_on="Matched Patent ID", right_on="Patent ID", how="left", suffixes=("_OCR", "_DB"))

# Save final output
#merged_df.to_excel("matched_patents.xlsx", index=False)

#print("Fuzzy matching completed. Results saved to 'matched_patents.xlsx'.")


c:\ProgramData\Anaconda3\envs\ada\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
merged_df

,Patent Type,Patent Category,Duration (Years),Patent Title,Patentee,Date Line,Extracted Date,Word Count,Character Count,Page Number,...,Matched Patent ID,Best Match,Match Score,Patent ID,Patentee name,Patent title,Issue date,Cleaned Patentee_DB,Cleaned Title_DB,Cleaned Date_DB
0,BREVET,INVENTION,CINQ,une nouvelle pompe aspirante et foulante propre,Cxauvis et GurrLormN,31 décembre 1818.,31 décembre 1818,667,3705,12,...,b1580,"(chauvin thomas, guillotin louis, nouvelle pom...",65.1,b1580,"Chauvin Thomas, Guillotin Louis","Nouvelle pompe foulante et aspirante, à incend...",31/12/1818,"chauvin thomas, guillotin louis","nouvelle pompe foulante et aspirante, à incend...",1818-12-31
1,BREVET,PERFECTIONNEMENT,DIX,des perfectionnemens apportés aux procédés de,J,. 28 janvier 1814.,28 janvier 1814,2831,15802,15,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BREVET,INVENTION,CINQ,des moyens de dételer les chevaux qui s’empor-,Laurent Joawne,25 février 1819,25 février 1819,2746,15381,25,...,b2199,"(joanne laurent, moyens de dételer les chevaux...",85.1,b2199,Joanne Laurent,"Moyens de dételer les chevaux qui s'emportent,...",25/02/1819,joanne laurent,"moyens de dételer les chevaux qui s'emportent,...",1819-02-25
3,BREVET,INVENTION,CINQ,des perfectionnemens apportés aux presses hy-,GrNormsre,"21 ,uxllel 1819.",Unrecognized Date,659,3638,34,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BREVET,IMPORTATION,QUINZE,une machine à rayer le papier,Decnanp,a1 juillet 1809.,Unrecognized Date,1915,10426,37,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,BREVET,INVENTION,CINQ,un moyen de tremper les lames de rasoirs,Derronmis,27 septembre 1822.,27 septembre 1822,318,1809,310,...,b1671,"(deffontis, procédés propres à tremper les lam...",70.6,b1671,Deffontis,Procédés propres à tremper les lames de rasoir...,27/09/1822,deffontis,procédés propres à tremper les lames de rasoir...,1822-09-27
68,BREVET,INVENTION,CINQ,des moyens de donner aux étoffes de laine,LacLoTTE,8 novembre 1822.,8 novembre 1822,216,1210,313,...,b616,"(laclotte, moyens propres à donner aux étoffes...",87.1,b616,Laclotte,"Moyens propres à donner aux étoffes de laine, ...",8/11/1822,laclotte,"moyens propres à donner aux étoffes de laine, ...",1822-11-08
69,BREVET,INVENTION,CINQ,un fusil dit & capote,Alexis Mortau,9 février 1821.,9 février 1821,228,1201,315,...,b307,"(moreau alexis, fabrique d'une arme à feu dite...",81.7,b307,Moreau Alexis,Fabrique d'une arme à feu dite à capote,8/02/1821,moreau alexis,fabrique d'une arme à feu dite à capote,1821-02-08
70,BREVET,IMPORTATION,QUINZE,deux machines propres,Unknown Patentee,10 décembre 1813.,10 décembre 1813,2378,13167,343,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Print only rows where "Matched Patent ID" is not null (successfully matched rows)
matched_rows = merged_df[merged_df["Matched Patent ID"].notna()]
matched_rows

,Patent Type,Patent Category,Duration (Years),Patent Title,Patentee,Date Line,Extracted Date,Word Count,Character Count,Page Number,...,Matched Patent ID,Best Match,Match Score,Patent ID,Patentee name,Patent title,Issue date,Cleaned Patentee_DB,Cleaned Title_DB,Cleaned Date_DB
0,BREVET,INVENTION,CINQ,une nouvelle pompe aspirante et foulante propre,Cxauvis et GurrLormN,31 décembre 1818.,31 décembre 1818,667,3705,12,...,b1580,"(chauvin thomas, guillotin louis, nouvelle pom...",65.1,b1580,"Chauvin Thomas, Guillotin Louis","Nouvelle pompe foulante et aspirante, à incend...",31/12/1818,"chauvin thomas, guillotin louis","nouvelle pompe foulante et aspirante, à incend...",1818-12-31
2,BREVET,INVENTION,CINQ,des moyens de dételer les chevaux qui s’empor-,Laurent Joawne,25 février 1819,25 février 1819,2746,15381,25,...,b2199,"(joanne laurent, moyens de dételer les chevaux...",85.1,b2199,Joanne Laurent,"Moyens de dételer les chevaux qui s'emportent,...",25/02/1819,joanne laurent,"moyens de dételer les chevaux qui s'emportent,...",1819-02-25
8,BREVET,INVENTION,CINQ,un nécessaire à jeu double et simple,Sarnr-Marrin,29 juin 1820.,29 juin 1820,245,1444,54,...,b1138,"(saint-martin, mécanisme double ou simple, que...",73.5,b1138,Saint-Martin,"Mécanisme double ou simple, que l'auteur nommé...",22/06/1820,saint-martin,"mécanisme double ou simple, que l'auteur nommé...",1820-06-22
9,BREVET,IMPORTATION,QUINZE,une machine à pulvériser les bois de teinture,Decrand,23 juin 1810.,23 juin 1810,918,4861,56,...,b217,"(degrand elzéard, machine à pulvériser les boi...",77.0,b217,Degrand Elzéard,Machine à pulvériser les bois de teinture,23/06/1810,degrand elzéard,machine à pulvériser les bois de teinture,1810-06-23
14,BREVET,INVENTION,CINQ,un appareil- distillatoire continu,Dérope,15 juillet 1820.,15 juillet 1820,2148,12126,77,...,b670,"(dérode nicolas, appareil distillatoire contin...",70.5,b670,Dérode Nicolas,Appareil distillatoire continu,15/07/1820,dérode nicolas,appareil distillatoire continu,1820-07-15
17,BREVET,PERFECTIONNEMENT,CINQ,une arme à feu perfectionnée,DEpousenT,22 septembre 1820.,22 septembre 1820,235,1304,88,...,b49,"(deboubert, arme à feu qui s'amorce et détonne...",64.9,b49,Deboubert,Arme à feu qui s'amorce et détonne au moyen de...,22/09/1820,deboubert,arme à feu qui s'amorce et détonne au moyen de...,1820-09-22
22,BREVET,INVENTION,QUINZE,un procédé propre à f'ah'riqùer l’acide sulfur...,Nicolas CzémenT,En,Unrecognized Date,1246,7251,99,...,b5,"(clément nicolas, procédés propres à la fabric...",67.8,b5,Clément Nicolas,Procédés propres à la fabrication de l'acide s...,4/11/1810,clément nicolas,procédés propres à la fabrication de l'acide s...,1810-11-04
27,BREVET,INVENTION,CINQ,une machine hydraulique,Carron,r. juin 1820.,Unrecognized Date,751,4118,113,...,b1086,"(capron, machine hydraulique appelée norpac, 1...",64.0,b1086,Capron,Machine hydraulique appelée Norpac,1/06/1820,capron,machine hydraulique appelée norpac,1820-06-01
30,BREVET,INVENTION,CINQ,un mécanisme applicable au piano,Roiter,14 août 1820.,14 août 1820,503,2883,178,...,b872,"(roller, mécanisme destiné à applanir la diffi...",65.5,b872,Roller,Mécanisme destiné à applanir la difficulté du ...,14/08/1820,roller,mécanisme destiné à applanir la difficulté du ...,1820-08-14
35,BREVET,INVENTION,QUINZE,diverses machines destinées à faconner les me-,James Wurre,4 mars 1811.,4 mars 1811,3546,19087,190,...,b514,"(white james, machine propre à suppléer à la m...",67.0,b514,White James,Machine propre à suppléer à la main-d'peuvre d...,4/03/1811,white james,machine propre à suppléer à la main-d'peuvre d...,1811-03-04
